<a href="https://colab.research.google.com/github/MogoWhite/Subtitle_Auto_Alignment_and_Translation/blob/main/Whisper_Timestamped_GPT%E5%AD%97%E5%B9%95%E8%87%AA%E5%8A%A8%E8%BD%B4%E7%BF%BB%E5%B7%A5%E5%85%B7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Whisper_Timestamped_GPT字幕自动轴翻工具
### 具体用来完成以下三个任务
1.   使用whisper的扩展库whisper_timestamped，进行语音转文字及单词级时间轴校准，并生成SRT格式的字幕文件。


2.   调整字幕文件（SRT格式）中字幕的起始和结束时间的功能：

    1.   将字幕的开始时间向前或向后移动。
    2.   将字幕的结束时间向后移动。
    3.   调整相邻字幕之间的时间间隔。


3.  调用api实现以下功能：
    1.   将字幕文件逐行翻译为目标语言。
    2.   支持生成双语字幕或仅翻译后的字幕。
    3.   支持导出为 ASS 或 SRT 格式。
    4.   增加多线程GPT处理。
    5.   增加多行输入，可以更好的联系上下文。
    6.   调整prompt，一定程度提高专有名词翻译精度。

4.  最后想了想，还是提取分行的文字直接扔到chatgpt翻译更方便，还免费。


---




*   首先使用下面的网站，从视频分离出mp3格式的音频。

       https://online-audio-converter.com


*   然后将音频，从本地拖拽到本页面左侧文件的位置，或使用文件栏左上角的**上传到会话存储空间**按钮上传文件。
![image.png](https://i.imgur.com/PoWwdAo.png)


*   接下来复制文件路径，用于输入后面的地址框。
![image.png](https://i.imgur.com/t0Lkqwr.png)

*  选择GPU
![image.png](https://i.imgur.com/CVhHPVC.png)




In [ ]:
#@title 安装运行库

!pip3 install ffmpeg
!pip3 install git+https://github.com/linto-ai/whisper-timestamped
!pip3 install onnxruntime torchaudio
!pip3 install pysrt
!pip3 install yt-dlp
!pip3 install openai  
!pip3 install pysubs2
!pip3 install tqdm

In [ ]:
#@title 测试 cuda 和 whisper_timestamped
import torch
import whisper_timestamped
print(torch.cuda.is_available())
help(whisper_timestamped.transcribe)

In [ ]:
#@title 加载whisper模型
import torch
import whisper_timestamped

model = whisper_timestamped.load_model("large-v2", device="cuda")

In [ ]:
#@title 运行whisper_timestamped 语音转文字 { display-mode: "form" }


# @markdown **要填！！！** input_file: 输入音频文件路径
# @markdown </br>
# @markdown </br> input_prompt: 第一句话的提示，影响第一句话的生成，默认留空
# @markdown </br>
# @markdown </br> length_penalty: 调节生成的句子的长度，值增大输出的句子有缩短倾向
# @markdown </br>
# @markdown </br> min_sentence_duration: 一句话持续时间的最小值


import pysrt
import torch
from datetime import timedelta

def timedelta_to_SubRipTime(td):
    """
    Converts a timedelta object to a SubRipTime object.
    
    Args:
    td (timedelta): A timedelta object representing a duration.

    Returns:
    pysrt.srttime.SubRipTime: A SubRipTime object representing the same duration as the input timedelta.
    """
    total_seconds = td.total_seconds()
    hours, remainder = divmod(total_seconds, 3600)
    minutes, seconds = divmod(remainder, 60)
    return pysrt.srttime.SubRipTime(int(hours), int(minutes), int(seconds), int(td.microseconds // 1000))

input_file = "/content/kuboasu.mp3" # @param {type:"string"}
audio = whisper_timestamped.load_audio(input_file)
input_prompt = "" # @param {type:"string"}

result = whisper_timestamped.transcribe(
    model=model,
    audio=audio,
    language='ja',
    task='transcribe',
    remove_punctuation_from_words=True,
    seed=0,
    vad=True,
    #detect_disfluencies=True,
    trust_whisper_timestamps=False,
    length_penalty=1 #@param {type:"slider", min:0.0, max:1.0, step:0.1}
    ,
    #长度惩罚因子
    initial_prompt=input_prompt,
    suppress_tokens='-1',
    min_word_duration=0.02 #@param {type:"slider", min:0.00, max:0.10, step:0.01}
    ,
    #compression_ratio_threshold=5.0,
    #logprob_threshold=-3.0
)

torch.cuda.empty_cache()

segments = result["segments"]
subtitles = pysrt.SubRipFile()

for data in segments:
    index = data["id"] + 1
    start = timedelta(seconds=data["start"])
    end = timedelta(seconds=data["end"])
    text = data["text"]
    sub = pysrt.SubRipItem(index=index,
                           start=timedelta_to_SubRipTime(start),
                           end=timedelta_to_SubRipTime(end),
                           text=text)

    subtitles.append(sub)

base_name = input_file.rsplit('.', 1)[0]
output_file = base_name + "_output.srt"
subtitles.save(output_file, encoding="utf-8")


In [ ]:
#@title 字幕调轴 { display-mode: "form" }

# @markdown </br>shift_start_ms: 开始时间轴上移动的毫秒数。负值表示字幕开始时间提前，正值表示字幕开始时间延后。
# @markdown </br>
# @markdown </br>shift_end_ms: 结束时间轴上移动的毫秒数。正值表示字幕结束时间延后，负值表示字幕结束时间提前。
# @markdown </br>
# @markdown </br>min_gap: 两个连续字幕之间的最小时间间隔。如果间隔小于此值，当前字幕的结束时间将延长以满足最小间隔要求。


input_srt_file = base_name + "_output.srt"
final_output_srt_file = base_name + "_adjusted.srt"
shift_start_ms = -10 #@param {type:"slider", min:-100, max:0, step:10}
shift_end_ms = 300 #@param {type:"slider", min:0, max:500, step:50}
min_gap = 300 #@param {type:"slider", min:0, max:500, step:50}


import pysrt
import datetime
from datetime import timedelta

def timedelta_to_subriptime(delta):
    """
    Converts a timedelta object to a SubRipTime object.
    
    Args:
    delta (timedelta): A timedelta object representing a duration.

    Returns:
    pysrt.SubRipTime: A SubRipTime object representing the same duration as the input timedelta.
    """
    return pysrt.SubRipTime(hours=delta.days * 24 + delta.seconds // 3600, minutes=(delta.seconds // 60) % 60, seconds=delta.seconds % 60, milliseconds=delta.microseconds // 1000)

def shift_start_time(subs, milliseconds):
    """
    Shifts the start times of subtitles by a given number of milliseconds.

    Args:
    subs (list): A list of SubRipItem objects representing subtitles.
    milliseconds (int): The number of milliseconds to shift the start times.

    Returns:
    list: A list of updated SubRipItem objects with shifted start times.
    """
    for i in range(len(subs)):
        if i == 0 or subs[i].start != subs[i - 1].end:
            new_start_time = subs[i].start + timedelta_to_subriptime(datetime.timedelta(milliseconds=milliseconds))
            if i > 0 and new_start_time < subs[i - 1].end:
                new_start_time = subs[i - 1].end
            subs[i].start = new_start_time
    return subs

def shift_end_time(subs, milliseconds):
    """
    Shifts the end times of subtitles by a given number of milliseconds.

    Args:
    subs (list): A list of SubRipItem objects representing subtitles.
    milliseconds (int): The number of milliseconds to shift the end times.

    Returns:
    list: A list of updated SubRipItem objects with shifted end times.
    """
    for i in range(len(subs) - 1):
        if subs[i].end != subs[i + 1].start:
            new_end_time = subs[i].end + timedelta_to_subriptime(datetime.timedelta(milliseconds=milliseconds))
            if new_end_time > subs[i + 1].start:
                new_end_time = subs[i + 1].start
            subs[i].end = new_end_time
    return subs

def adjust_subtitle_timing(subs, min_gap=300):
    """
    Adjusts the timing of subtitles to ensure a minimum gap between them.

    Args:
    subs (list): A list of SubRipItem objects representing subtitles.
    min_gap (int, optional): The minimum gap in milliseconds between subtitles. Defaults to 300.

    Returns:
    list: A
    for i in range(len(subs) - 1):
        current_sub = subs[i]
        next_sub = subs[i + 1]
        gap = next_sub.start - current_sub.end

        if 0 < gap < min_gap:
            current_sub.end = next_sub.start

    return subs

def process_subtitle_file(input_srt_file, output_srt_file, shift_start_ms, shift_end_ms, min_gap):
    """
    Processes a subtitle file by shifting start and end times and adjusting the timing between subtitles.

    Args:
    input_srt_file (str): The input subtitle file (SRT format).
    output_srt_file (str): The output subtitle file with modified timings (SRT format).
    shift_start_ms (int): The number of milliseconds to shift the start times.
    shift_end_ms (int): The number of milliseconds to shift the end times.
    min_gap (int): The minimum

    subs = pysrt.open(input_srt_file)

    subs = shift_start_time(subs, shift_start_ms)

    subs = shift_end_time(subs, shift_end_ms)

    subs = adjust_subtitle_timing(subs, min_gap)

    subs.save(output_srt_file, encoding='utf-8')

# 调用函数处理字幕文件
# ... (代码开头部分保持不变)

process_subtitle_file(input_srt_file, final_output_srt_file, shift_start_ms, shift_end_ms, min_gap)

In [ ]:
#@title 可跳过！！！ SRT字幕文件转ASS字幕文件 附加字幕样式

# @markdown style_type: 字幕组样式



style_type = "石森璃花" # @param ["井上和","石森璃花"]
input_srt = final_output_srt_file
output_ass_file = base_name + "_output.ass"

def srt_to_ass(input_srt, output_ass, style_type):
    with open(input_srt, 'r', encoding='utf-8') as f:
        srt_contents = f.read()

    srt_lines = srt_contents.split('\n')
    ass_dialogues = []

    if style_type == '井上和':
        ass_header = (
            "[Script Info]\n"
            "; Script generated by Aegisub 9212-dev-3a38bf16a\n"
            "; http://www.aegisub.org/\n"
            "Title: Default Aegisub file\n"
            "ScriptType: v4.00+\n"
            "WrapStyle: 0\n"
            "ScaledBorderAndShadow: yes\n"
            "YCbCr Matrix: None\n"
            "PlayResX: 1920\n"
            "PlayResY: 1080\n"
            "\n"
            "[Aegisub Project Garbage]\n"
            "Last Style Storage: hinaai\n"
            "Video Zoom Percent: 0.875000\n"
            "Scroll Position: 63\n"
            "\n"
            "[V4+ Styles]\n"
            "Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding\n"
            "Style: 井上和,思源黑体 CN,80,&H00FFFFFF,&H000000FF,&H030000FF,&HFF000000,-1,0,0,0,100,100,1.5,0,1,3,0,2,10,10,10,1\n"
            "Style: staff,思源黑体 CN,55,&H00FFFFFF,&H00FFFFFF,&H34000000,&H00000000,-1,0,0,0,100,100,3,0,1,2.5,0,7,16,13,4,1\n"
            "Style: 水印,悦黑 - yolan,30,&H73FFFFFF,&HFAFFFFFF,&H00FFFFFF,&HD1000300,-1,0,0,0,104,105,5,0,1,0,1.6,9,11,11,11,1\n"
            "Style: 屏字,思源黑体 CN,70,&H00FFFFFF,&H000019FF,&H00000000,&H00000000,-1,0,0,0,100,100,2,0,1,3,0,2,11,11,11,1\n"
            "Style: 上注释,思源黑体 CN,55,&H00FFFFFF,&H000019FF,&H1E000000,&H9E000000,-1,0,0,0,100,100,3,0,1,3,2,8,11,11,11,1\n"
            "Style: 竖,@思源黑体 CN,190,&H00FFFFFF,&H000000FF,&H00000000,&H00000000,-1,0,0,0,100,100,10,-90,1,3,0,5,11,11,11,1\n"
            "\n"
            "[Events]\n"
            "Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text\n"
        )
    else:  # style_type == 'sslh'
        ass_header = (
            "[Script Info]\n"
            "; Script generated by Aegisub 9212-dev-3a38bf16a\n"
            "; http://www.aegisub.org/\n"
            "Title: Default Aegisub file\n"
            "ScriptType: v4.00+\n"
            "WrapStyle: 0\n"
            "ScaledBorderAndShadow: yes\n"
            "YCbCr Matrix: None\n"
            "PlayResX: 1920\n"
            "PlayResY: 1080\n"
            "\n"
            "[Aegisub Project Garbage]\n"
            "Last Style Storage: 璃花\n"
            "Video Zoom Percent: 0.500000\n"
            "Active Line: 1\n"
            "\n"
            "[V4+ Styles]\n"
            "Format: Name, Fontname, Fontsize, PrimaryColour, SecondaryColour, OutlineColour, BackColour, Bold, Italic, Underline, StrikeOut, ScaleX, ScaleY, Spacing, Angle, BorderStyle, Outline, Shadow, Alignment, MarginL, MarginR, MarginV, Encoding\n"
            "Style: 对话 1080P,思源黑体 CN Medium,75,&H00FFFFFF,&H000000FF,&H00EE49FA,&H002BE257,-1,0,0,0,100,100,4,0,1,4,3,2,10,10,10,1\n"
            "Style: staff 1080P,方正准圆简体,45,&H00FFFFFF,&H000000FF,&H00EE49FA,&H002BE257,-1,0,0,0,100,100,4,0,1,4,3,7,15,15,10,1\n"
            "Style: 屏字 1080P,方正兰亭圆_GBK,60,&H00FFFFFF,&H000019FF,&H00000000,&H00000000,-1,0,0,0,100,100,4,0,1,4,0,5,10,10,10,1\n"
            "Style: 注释 1080P,等线,50,&H00FFFFFF,&H000000FF,&H00F67DFF,&H002BE257,-1,0,0,0,100,100,4,0,1,4,3,8,10,10,15,1\n"
            "Style: 对话 720P,思源黑体 CN Medium,50,&H00FFFFFF,&H000000FF,&H00EE49FA,&H002BE257,-1,0,0,0,100,100,4,0,1,4,3,2,10,10,10,1\n"
            "Style: staff 720P,方正准圆简体,30,&H00FFFFFF,&H000000FF,&H00EE49FA,&H002BE257,-1,0,0,0,100,100,4,0,1,4,3,7,15,15,10,1\n"
            "Style: 屏字 720P,方正兰亭圆_GBK,40,&H00FFFFFF,&H000019FF,&H00000000,&H00000000,-1,0,0,0,100,100,4,0,1,4,0,5,10,10,10,1\n"
            "Style: 注释 720P,等线,33,&H00FFFFFF,&H000000FF,&H00F67DFF,&H002BE257,-1,0,0,0,100,100,4,0,1,4,3,8,10,10,15,1\n"
            "\n"
            "[Events]\n"
            "Format: Layer, Start, End, Style, Name, MarginL, MarginR, MarginV, Effect, Text\n"
        )

    for i in range(0, len(srt_lines), 4):
        if srt_lines[i].isdigit():
            start_time, end_time = srt_lines[i+1].split(" --> ")
            ass_start_time = start_time.replace(",", ".")
            ass_end_time = end_time.replace(",", ".")

            dialogue_text = srt_lines[i+2].replace("\\N", "\n")
            dialogue_line = f"Dialogue: 0,{ass_start_time},{ass_end_time},Default,,0,0,0,,{dialogue_text}"
            ass_dialogues.append(dialogue_line)

    with open(output_ass, 'w', encoding='utf-8') as f:
        f.write(ass_header)
        f.write('\n'.join(ass_dialogues))

# 使用函数将 SRT 文件转换为 ASS 文件

srt_to_ass(input_srt, output_ass_file, style_type)

In [ ]:
#@title （可选，没有api，想使用免费翻译，请跳过这一段）并行GPT处理  { display-mode: "form" }


# @markdown **要填！！！adjusted后缀的字幕文件地址**  sub_path: 待翻译的字幕文件路径（字符串）
# @markdown </br>
# @markdown </br>**要填！！！**openai_key: OpenAI API 密钥（字符串）
# @markdown </br>
# @markdown </br>target_language: 目标语言，例如 "zh-hans"（简体中文）或 "english"（英语）
# @markdown </br>
# @markdown </br>content_related_text: GPT-4 用于翻译的提示，如专有名词的翻译方式等，现在只对第一批输入的行有效，例：请把 あすか 翻译为 飞鸟.（字符串）
# @markdown </br>
# @markdown </br>temperature: GPT-4 API 的温度值，控制输出结果的随机性（0 到 1.0 之间的浮点数）
# @markdown </br>
# @markdown </br>output_mode: 输出模式，"translated"（仅翻译后的字幕）或 "bilingual"（双语字幕）
# @markdown </br>
# @markdown </br>model_name: GPT模型，'gpt-3.5-turbo'或'gpt-4'
# @markdown </br>
# @markdown </br>output_format: 输出文件格式，"ass"（ASS 字幕）或 "srt"（SRT 字幕）
# @markdown </br>
# @markdown </br>batch_size: 每次输入的行数，增大每行之间内容的关联性增加，但增加输出出错的可能性，建议为全部行数的1/2或1/4，不要超过50（1 到 100 之间的整数）


"""
使用 GPT-4 API 翻译字幕文件

功能：
    1. 将字幕文件逐行翻译为目标语言。
    2. 支持生成双语字幕或仅翻译后的字幕。
    3. 支持导出为 ASS 或 SRT 格式。
    4. 增加多线程GPT处理。
    5. 增加多行输入，可以更好的联系上下文。

输入：
    sub_path：待翻译的字幕文件路径（字符串）。
    openai_key：OpenAI API 密钥（字符串）。
    target_language：目标语言，例如 "zh-hans"（简体中文）或 "english"（英语）。
    content_related_text：GPT-4 用于翻译的提示（字符串）。
    temperature：GPT-4 API 的温度值，控制输出结果的随机性（0 到 1.0 之间的浮点数）。
    output_mode：输出模式，"translated"（仅翻译后的字幕）或 "bilingual"（双语字幕）。
    model_name: GPT模型，'gpt-3.5-turbo'或'gpt-4'
    output_format：输出文件格式，"ass"（ASS 字幕）或 "srt"（SRT 字幕）。
    batch_size：每次输入的行数（10 到 100 之间的整数）

输出：
    翻译后的字幕文件（ASS 或 SRT 格式）。

示例：
    sub_path = '/content/output_adjusted.srt'
    openai_key = 'your_openai_api_key'
    target_language = 'zh-hans'
    content_related_text = "You are a language expert. ..."
    temperature = 0.6
    output_mode= 'bilingual'
    model_name= 'gpt-3.5-turbo'
    output_format = "srt"
    batch_size=1

    运行脚本后，会生成一个双语字幕文件 "output_adjusted_bilingual_translation.srt"。
"""

sub_path = '/content/kuboasu_output.ass'  # @param {type:"string"}
openai_key = ''  # @param {type:"string"}
target_language = 'zh-hans'  # @param ["zh-hans","english"]

prompt = "You are a language expert. Your task is to translate the input subtitle text, sentence by sentence, into the user specified target language. However, please utilize the context to improve the accuracy and quality of translation. Be aware that the input text may contain typos and grammar mistakes; utilize the context to correct the translation. Please return only translated content and do not include the original text. Preserve the \"|||\" separator in the output. Do not use any punctuation around the returned text. Do not translate people's names and leave them in the original language." # @param {type:"string"}

temperature = 0.6  # @param {type:"slider", min:0, max:1.0, step:0.1}
output_mode= 'translated' # @param ["translated","bilingual"]
model_name= 'gpt-3.5-turbo' # @param ["gpt-4","gpt-3.5-turbo"]
output_format = "srt"  # @param ["ass","srt"]
batch_size=100 #@param {type:"slider", min:10, max:100, step:1}

import sys
import os
import re
import time
import codecs
from pathlib import Path
from tqdm import tqdm
from google.colab import files
from IPython.display import clear_output
from concurrent.futures import ThreadPoolExecutor, as_completed


sub_name = sub_path
sub_basename = Path(sub_name).stem

import openai
import pysubs2

clear_output()

class ChatGPTAPI():
    def __init__(self, key, language, prompt, temperature, model):
        """
        Initializes the ChatGPTAPI class with the given API key, target language, prompt, temperature, and model name.

        Args:
        key (str): The API key for OpenAI.
        language (str): The target language for translation.
        prompt (str): The system prompt to set the context for translation.
        temperature (float): The temperature for controlling randomness during sampling.
        model_name (str): The name of the GPT model to use for translation.
        """
        self.key = key
        self.language = language
        self.prompt = prompt
        self.temperature = temperature
        self.model = model_name

    def translate(self, text):
        """
        Translates the given text to the target language using the OpenAI GPT model.

        Args:
        text (str): The input text to be translated.

        Returns:
        str: The translated text in the target language.
        """
        openai.api_key = self.key
        try:
            completion = openai.ChatCompletion.create(
                model=self.model,
                messages=[
                    {
                        "role": "system",
                        "content": f'{self.prompt}'
                    },
                    {
                        "role":"user",
                        "content": f"Original text:`{text}`. Target language: {self.language}"
                    }
                ],
                temperature=self.temperature
            )
            t_text = (
                completion["choices"][0]
                .get("message")
                .get("content")
                .encode("utf8")
                .decode()
            )
        except Exception as e:
            sleep_time = int(60)
            time.sleep(sleep_time)
            print(e, f"will sleep  {sleep_time} seconds")
            openai.api_key = self.key
            completion = openai.ChatCompletion.create(
                model=self.model,
                messages=[
                    {
                        "role": "system",
                        "content": f'{self.prompt}'
                    },
                    {
                        "role": "user",
                        "content": f"Original text:`{text}`. Target language: {self.language}"
                    }
                ],
                temperature=self.temperature
            )
            t_text = (
                completion["choices"][0]
                .get("message")
                .get("content")
                .encode("utf8")
                .decode()
            )
        return t_text

class SubtitleTranslator():
    def __init__(self, sub_src, model, key, language, prompt, temperature, output_mode='bilingual'):
        """
        Initializes the SubtitleTranslator class with the given subtitle source file, translation model, API key, target language, system prompt, temperature, and output mode.

        Args:
        sub_src (str): The path to the subtitle source file (SRT or ASS format).
        model (class): The translation model class.
        key (str): The API key for the translation model.
        language (str): The target language for translation.
        prompt (str): The system prompt to set the context for translation.
        temperature (float): The temperature for controlling randomness during sampling.
        output_mode (str): The mode for outputting translated subtitles ('bilingual' or 'translated'). Default is 'bilingual'.
        """
        self.sub_src = sub_src
        self.translate_model = model(key, language, prompt, temperature, model)
        self.translations = []
        self.output_mode = output_mode

    def translate_by_line(self):
        """
        Translates the subtitle lines using the translation model and generates a new subtitle file with translations.

        Returns:
        tuple: A tuple containing the translated subtitle file and a list of translated lines.
        """
        sub_trans = pysubs2.load(self.sub_src)
        total_lines = len(sub_trans)
        lines_to_translate = []

        # 新增一个列表，用于保存任务
        tasks = []

        with ThreadPoolExecutor() as executor:
            with tqdm(total=total_lines, ncols=80) as pbar:
              for i, line in enumerate(sub_trans):
                  lines_to_translate.append(line.text)

                  if (i + 1) % batch_size == 0 or i == total_lines - 1:
                      combined_text = '|||'.join(lines_to_translate)
                      # 将任务提交到线程池，并将任务添加到tasks列表
                      task = executor.submit(self.translate_model.translate, combined_text)
                      tasks.append((task, i, lines_to_translate))
                      lines_to_translate = []

              # 在线程池中处理任务
              for task, i, lines_to_translate in tasks:
                  try:
                      translated_text = task.result()
                      translated_lines = translated_text.split('|||')
                      pbar.write(str(translated_lines))

                      for j, t_line in enumerate(translated_lines):
                          if self.output_mode == 'bilingual':
                              sub_trans[i - len(lines_to_translate) + 1 + j].text += (r'\N' + t_line)
                          elif self.output_mode == 'translated':
                              sub_trans[i - len(lines_to_translate) + 1 + j].text = t_line
                          self.translations.append(t_line)
                      # 更新进度条
                      pbar.update(len(lines_to_translate))
                      pbar.refresh()
                  except Exception as e:
                      print(f"Error occurred during translation: {e}")
                      pbar.update(len(lines_to_translate))
            return sub_trans, self.translations



translate_model = ChatGPTAPI

assert translate_model is not None, "unsupported model"
OPENAI_API_KEY = openai_key

if not OPENAI_API_KEY:
    raise Exception(
        "OpenAI API key not provided, please google how to obtain it"
    )

t = SubtitleTranslator(
    sub_src=sub_name,
    model=translate_model,
    key=OPENAI_API_KEY,
    language=target_language,
    prompt=prompt,
    temperature=temperature,
    output_mode=output_mode)

translation, _ = t.translate_by_line()

if output_format == 'ass':
    translation.save(sub_basename +'_'+ output_mode +'_translation.ass')
    files.download(sub_basename +'_'+ output_mode + '_translation.ass')
elif output_format == 'srt':
    translation.save(sub_basename +'_'+ output_mode + '_translation.srt')
    files.download(sub_basename +'_'+ output_mode + '_translation.srt')

print('双语字幕生成完毕 All done!')


100%|███████████████████████████████████████████| 56/56 [01:34<00:00,  1.69s/it]

['明天香的写真集博物馆', '哎呀！', '哇，好漂亮', '马上就很漂亮', '最喜欢的剪辑是从一开始就喜欢的全男性', '哎呀，嘴巴好可爱', '因为嘴巴很可爱', 'T恤', '成年明天香的T恤', '啊，看起来好美味', '哎呀', '刷牙', '明天香果然嘴巴可爱', '明天香穿着休闲装', '露出额头的明天香', '新鲜', '好', '哎呀，这顶帽子真的很适合', '为什么？', '哇', '这套牛仔装超级可爱', '哎呀', '哇，好瘦', '只穿着牛仔衬衫', '啊，这是你说的第一款耳环', '浓重的唇膏最棒', '哎呀', '锁骨好漂亮', '超级漂亮', '感觉好像要被明天香生气', '明天香变成大人了', '明天香不要生气', '你看，三期生和明天香一起', '从十几岁开始就一直在一起', '变得很高兴', '明天香不要生气', '吃着冰淇淋', '啊，好可爱', '啊哈哈', '啊，但是也有孩子气的部分', '很高兴', '哇，眼睛的颜色', '好漂亮', '光线进入', '这看完就回来', '看标题博物馆', '哈', '这一本好棒', '明天香都被塞满了', '感觉像是调色盘的明天香', '喜欢什么颜色的明天香？', '那种感觉', '全部都可爱', '最棒的博物馆', '好可爱']


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

双语字幕生成完毕 All done!


In [ ]:
#@title 字幕文字分行提取
import pysubs2
import ipywidgets as widgets
from IPython.display import display

def print_subtitles_with_separator(subtitle_file, lines_per_widget=25):
    sub_data = pysubs2.load(subtitle_file)
    separated_texts = []

    for i in range(0, len(sub_data), lines_per_widget):
        separated_lines = []
        prompt = "请将以下日语字幕文本逐句翻译成中文："
        for line in sub_data[i:i + lines_per_widget]:
            separated_lines.append(line.text)
        combined_text = '|||'.join(separated_lines)
        separated_texts.append(prompt + combined_text + "。请利用上下文提高翻译的准确性和质量。请注意，输入文本可能包含错别字和语法错误；利用上下文来纠正翻译。请仅返回翻译后的内容，不要包含原始文本。请保留输出中的'|||'分隔符。不要在返回的文本中使用任何标点符号。不要翻译人名，保留原始语言。")
    return separated_texts

# 使用您的字幕文件路径替换 '/content/misa_output.ass'
# 如果你要使用 SRT 文件，只需更改文件路径
subtitle_file = '/content/output_adjusted.srt'  # @param {type:"string"}

# 修改这个值以更改每个文本窗口的行数
lines_per_widget = 40  # @param {type:"integer"}

separated_texts = print_subtitles_with_separator(subtitle_file, lines_per_widget)

for idx, separated_text in enumerate(separated_texts):
    # 创建一个 Text widget
    text_widget = widgets.Textarea(
        value=separated_text,
        description=f'Output {idx + 1}:',
        layout=widgets.Layout(width='100%', height='200px')
    )

    # 显示 Text widget
    display(text_widget)


**上面的一段一段扔到chatgpt里：**


In [ ]:
import pysubs2

def write_subtitles_from_separated_text(subtitle_file, separated_text, output_mode="translated"):
    sub_data = pysubs2.load(subtitle_file)
    split_lines = separated_text.split('|||')

    if len(sub_data) != len(split_lines):
        print("Warning: Number of lines in the original subtitle file and separated text do not match.")
        return

    for i, line in enumerate(sub_data):
        if output_mode == "bilingual":
            line.text += r'\N' + split_lines[i]
        elif output_mode == "translated":
            line.text = split_lines[i]

    sub_data.save(subtitle_file)

# 使用您的字幕文件路径替换 'your_subtitle_file.ass'
subtitle_file = '/content/enasu_output.ass' # @param {type:"string"}

# 用 '|||' 连接的文本字符串
separated_text = ' '  # @param {type:"string"}

# 设置 output_mode 为 "translated" 或 "bilingual"
output_mode = "translated"  # @param ["translated", "bilingual"]

write_subtitles_from_separated_text(subtitle_file, separated_text, output_mode)
